In [1]:
!pip install patool

import patoolib
import pandas as pd
import glob
import zipfile
import os
from tqdm import tqdm


rar_path = 'data/source/train_dataset_rzhd_model_train.rar'  # путь к твоему .rar файлу

output_folder = '/unzipped_folder'
os.makedirs(output_folder, exist_ok=True)
patoolib.extract_archive(rar_path, outdir='')


output_folder = 'unzipped_folder'

# Путь к папке с файлами
file_path = 'РЖД train/Выгрузка_маркетинговые списки/*.xls'

# Список всех файлов
all_files = glob.glob(file_path)

# Создание списка DataFrame для каждого файла
df_list = []

for file in all_files:
    # Чтение файла
    df = pd.read_excel(file)

    # Извлечение области из имени файла
    region = file.split('_')[-1].split('.')[0]  # Например, "Владимирская область"

    # Добавление столбца с регионом
    df['Регион'] = region

    # Добавление в список
    df_list.append(df)

# Объединение всех DataFrame в один
df_mc = pd.concat(df_list, ignore_index=True)

df_raw = pd.read_excel('РЖД train/Выгрузки_интересы+обращения+объёмы перевозок/Объёмы перевозок.xls', skiprows=1)

list_vol = list()
list_price = list()
unnamed_list = ['Unnamed: 0',	'Unnamed: 1',	'Unnamed: 2', 	'Unnamed: 3', 'Месяц']
for col in df_raw.columns:
  if col not in unnamed_list:
    if '.1' in col:
      list_vol.append(col)
    else:
      list_price.append(col)

df_vol = df_raw[unnamed_list + list_vol]
df_price = df_raw[unnamed_list + list_price]

df_vol.rename(columns={'Unnamed: 0': 'ID',
                       'Unnamed: 1': 'Субъект федерации отп',
                       'Unnamed: 2': 'Субъект федерации наз',
                       'Unnamed: 3': 'Код груза',
                       'Месяц': 'Гр груза по опер.номен'}, inplace=True)
df_price.rename(columns={'Unnamed: 0': 'ID',
                       'Unnamed: 1': 'Субъект федерации отп',
                       'Unnamed: 2': 'Субъект федерации наз',
                       'Unnamed: 3': 'Код груза',
                       'Месяц': 'Гр груза по опер.номен' }, inplace=True)

df_vol = df_vol.iloc[1:]
df_price = df_price.iloc[1:]
df_vol.columns = df_vol.columns.str.replace('.1$', '', regex=True)


df_req = pd.read_excel('РЖД train/Выгрузки_интересы+обращения+объёмы перевозок/Обращения.xls')

df_interest = pd.read_excel('РЖД train/Выгрузки_интересы+обращения+объёмы перевозок/Интересы.xls')

date_columns = [col for col in df_price.columns if '/' in col]
date_columns = pd.to_datetime(date_columns, format='%Y/%m')
sorted_date_columns = sorted(date_columns)

# Обновление DataFrame с новыми отсортированными столбцами
sorted_columns = df_price.columns.tolist()
sorted_columns = [col for col in sorted_columns if col not in date_columns]
sorted_columns += [d.strftime('%Y/%m') for d in sorted_date_columns]

# Обновляем DataFrame
df_price = df_price.reindex(columns=sorted_columns)


date_columns = [col for col in df_vol.columns if '/' in col]
date_columns = pd.to_datetime(date_columns, format='%Y/%m')
sorted_date_columns = sorted(date_columns)

# Обновление DataFrame с новыми отсортированными столбцами
sorted_columns = df_vol.columns.tolist()
sorted_columns = [col for col in sorted_columns if col not in date_columns]
sorted_columns += [d.strftime('%Y/%m') for d in sorted_date_columns]

# Обновляем DataFrame
df_vol = df_vol.reindex(columns=sorted_columns)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.6 MB/s eta 0:00:00


INFO patool: Extracting /content/drive/MyDrive/digital/train_dataset_rzhd_model_train.rar ...
INFO:patool:Extracting /content/drive/MyDrive/digital/train_dataset_rzhd_model_train.rar ...


Mounted at /content/drive


INFO patool: running /usr/bin/unrar x -- /content/drive/MyDrive/digital/train_dataset_rzhd_model_train.rar
INFO:patool:running /usr/bin/unrar x -- /content/drive/MyDrive/digital/train_dataset_rzhd_model_train.rar
INFO patool:     with cwd='/content', input=''
INFO:patool:    with cwd='/content', input=''
INFO patool: ... /content/drive/MyDrive/digital/train_dataset_rzhd_model_train.rar extracted to `/content'.
INFO:patool:... /content/drive/MyDrive/digital/train_dataset_rzhd_model_train.rar extracted to `/content'.
<ipython-input-1-3c1093c6d82f>:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vol.rename(columns={'Unnamed: 0': 'ID',
<ipython-input-1-3c1093c6d82f>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: htt

In [2]:
df_vol.to_csv('df_vol.csv')
df_price.to_csv('df_price.csv')
df_mc.to_csv('df_mc.csv')
df_interest.to_csv('df_interest.csv')
df_req.to_csv('df_rew.csv')

In [3]:
df_req.to_csv('df_req.csv')